In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import os
from itertools import product

class LSTMModel:
    """
    A wrapper class for LSTM Regression model with enhanced functionality
    """
    def __init__(self, 
                 input_shape=None, 
                 units=50, 
                 dropout_rate=0.2, 
                 learning_rate=0.001):
        """
        Initialize the LSTM model with configurable parameters

        Parameters:
        - input_shape: Shape of input data (time_steps, features)
        - units: Number of LSTM units
        - dropout_rate: Dropout rate for regularization
        - learning_rate: Learning rate for Adam optimizer
        """
        self.input_shape = input_shape
        self.units = units
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.model = None
        self.history = None
        self.best_params = None

    def _create_sequences(self, X, y, time_steps=10):
        """
        Transform data into sequences suitable for LSTM
        
        Parameters:
        X (array): Features
        y (array): Target
        time_steps (int): Number of time steps in each sequence
        
        Returns:
        X_seq, y_seq: Data transformed into sequences
        """
        X_seq, y_seq = [], []
        for i in range(len(X) - time_steps):
            X_seq.append(X[i:i + time_steps])
            y_seq.append(y[i + time_steps])
        return np.array(X_seq), np.array(y_seq)

    def build_model(self):
        """
        Build the LSTM model architecture
        
        Returns:
        Compiled Keras LSTM model
        """
        if self.input_shape is None:
            raise ValueError("Input shape must be set before building the model")

        model = Sequential()
        
        # First LSTM layer with return sequences for stacking
        model.add(LSTM(units=self.units, 
                      return_sequences=True, 
                      input_shape=self.input_shape, 
                      recurrent_dropout=self.dropout_rate/2))
        model.add(Dropout(self.dropout_rate))
        
        # Second LSTM layer
        model.add(LSTM(units=self.units//2, 
                      return_sequences=False, 
                      recurrent_dropout=self.dropout_rate/2))
        model.add(Dropout(self.dropout_rate))
        
        # Output layer
        model.add(Dense(1))
        
        # Compile model
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), 
                     loss='mse', 
                     metrics=['mae'])
        
        return model

    def fit(self, X_train, y_train, X_val=None, y_val=None, time_steps=10, batch_size=32, epochs=100):
        """
        Train the LSTM model

        Parameters:
        - X_train: Training features
        - y_train: Training target values
        - X_val: Validation features (optional)
        - y_val: Validation target values (optional)
        - time_steps: Number of time steps for sequences
        - batch_size: Training batch size
        - epochs: Number of training epochs
        """
        # Ensure input is numpy array
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1, 1)

        # Create sequences
        X_train_seq, y_train_seq = self._create_sequences(X_train, y_train, time_steps)
        
        # Set input shape if not already set
        if self.input_shape is None:
            self.input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
        
        # Build model
        self.model = self.build_model()
        
        # Prepare validation data
        validation_data = None
        if X_val is not None and y_val is not None:
            X_val = np.array(X_val)
            y_val = np.array(y_val).reshape(-1, 1)
            X_val_seq, y_val_seq = self._create_sequences(X_val, y_val, time_steps)
            validation_data = (X_val_seq, y_val_seq)
        
        # Define callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001),
            ModelCheckpoint('models/best_lstm_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
        ]
        
        # Train the model
        self.history = self.model.fit(
            X_train_seq, y_train_seq,
            validation_data=validation_data,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )
        
        return self

    def predict(self, X_test, time_steps=10):
        """
        Make predictions using the trained model

        Parameters:
        - X_test: Test features
        - time_steps: Number of time steps for sequences

        Returns:
        Predictions
        """
        if self.model is None:
            raise ValueError("Model must be trained before making predictions")
        
        X_test = np.array(X_test)
        X_test_seq, _ = self._create_sequences(X_test, np.zeros(len(X_test)), time_steps)
        return self.model.predict(X_test_seq)

    def evaluate(self, X_test, y_test, time_steps=10):
        """
        Evaluate model performance

        Parameters:
        - X_test: Test features
        - y_test: True target values
        - time_steps: Number of time steps for sequences

        Returns:
        Dictionary of performance metrics
        """
        X_test = np.array(X_test)
        y_test = np.array(y_test).reshape(-1, 1)
        
        X_test_seq, y_test_seq = self._create_sequences(X_test, y_test, time_steps)
        y_pred = self.predict(X_test, time_steps)
        
        return {
            'mse': mean_squared_error(y_test_seq, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test_seq, y_pred)),
            'mae': mean_absolute_error(y_test_seq, y_pred),
            'r2': r2_score(y_test_seq, y_pred)
        }

    def optimize_hyperparameters(self, X_train, y_train, X_val, y_val, time_steps=10):
        """
        Perform hyperparameter optimization

        Parameters:
        - X_train, y_train: Training data
        - X_val, y_val: Validation data
        - time_steps: Number of time steps for sequences

        Returns:
        Best hyperparameters
        """
        # Prepare sequences
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1, 1)
        X_val = np.array(X_val)
        y_val = np.array(y_val).reshape(-1, 1)
        
        X_train_seq, y_train_seq = self._create_sequences(X_train, y_train, time_steps)
        X_val_seq, y_val_seq = self._create_sequences(X_val, y_val, time_steps)
        
        # Define hyperparameters for optimization
        hyperparams = {
            'units': [32, 64, 128],
            'learning_rate': [0.001, 0.0005],
            'dropout_rate': [0.2, 0.3],
            'batch_size': [32, 64]
        }
        
        best_params = None
        best_val_loss = float('inf')
        
        # Generate all combinations of hyperparameters
        param_combinations = list(product(
            hyperparams['units'],
            hyperparams['learning_rate'],
            hyperparams['dropout_rate'],
            hyperparams['batch_size']
        ))
        
        for params in param_combinations:
            units, learning_rate, dropout_rate, batch_size = params
            
            # Temporarily set model parameters
            self.units = units
            self.learning_rate = learning_rate
            self.dropout_rate = dropout_rate
            self.input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
            
            # Build and train model
            self.model = self.build_model()
            history = self.model.fit(
                X_train_seq, y_train_seq,
                validation_data=(X_val_seq, y_val_seq),
                epochs=10,
                batch_size=batch_size,
                verbose=0
            )
            
            # Get validation loss
            val_loss = history.history['val_loss'][-1]
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_params = {
                    'units': units,
                    'learning_rate': learning_rate,
                    'dropout_rate': dropout_rate,
                    'batch_size': batch_size
                }
        
        # Set best parameters
        self.best_params = best_params
        self.units = best_params['units']
        self.learning_rate = best_params['learning_rate']
        self.dropout_rate = best_params['dropout_rate']
        
        return best_params

    def save(self, filepath='models/lstm_model.h5'):
        """
        Save the trained model

        Parameters:
        - filepath: Path to save the model
        """
        if not os.path.exists('models'):
            os.makedirs('models')
        self.model.save(filepath)

    @classmethod
    def load(cls, filepath):
        """
        Load a pre-trained model

        Parameters:
        - filepath: Path to the saved model

        Returns:
        Loaded model
        """
        return tf.keras.models.load_model(filepath)

# Utility function for backward compatibility
def train_lstm_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, time_steps=20):
    """
    Convenience function to train an LSTM model

    Parameters:
    - X_train_scaled, y_train_scaled: Scaled training data
    - X_test_scaled, y_test_scaled: Scaled testing data
    - time_steps: Number of time steps for sequences

    Returns:
    best_model, history, X_test_seq, y_test_seq, best_params
    """
    # Initialize and train LSTM model
    lstm_model = LSTMModel()
    lstm_model.fit(
        X_train_scaled, y_train_scaled, 
        X_test_scaled, y_test_scaled, 
        time_steps=time_steps
    )
    
    # Prepare test sequences for return
    X_test_scaled = np.array(X_test_scaled)
    y_test_scaled = np.array(y_test_scaled).reshape(-1, 1)
    X_test_seq, y_test_seq = lstm_model._create_sequences(X_test_scaled, y_test_scaled, time_steps)
    
    return (
        lstm_model.model, 
        lstm_model.history, 
        X_test_seq, 
        y_test_seq, 
        lstm_model.best_params or {}
    )